# 03 - Preparación de los datos
## Trabajo MDS - 2o. Trim. ==> Inmigracion en Chile
`Este notebook contiene las transformaciones que hayan sido necesarias para el algoritmo de machine learning`

**Profesor:** Donoso I.<br>
**Alumnos:** Briceno, H.; Rovai, M.; Tessada G.
<p>Santiago, 06 de Noviembre 2018

<p><img src="./images/03_prep_data.png"></p>

---
## Pregunta:
### En cuales comunas se concentran los inmigrantes que llegaron a la Región Metropolitana? Quien son? Como són? Y los nuevos, donde viverán?
---

**Importación de las principales librerias**

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline  

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

import warnings
warnings.filterwarnings("ignore")

### 1. Importar el dataset

En esta etapa del proyecto, analizaremos datos de los extranjeros en la Región Metropilitana. Los datos fueron obtenídos a partir de los datos crudos de: [Departamento de Extranjeria y Migracion de Chile](http://www.extranjeria.gob.cl/estadisticas-migratorias/). 

La base de datos "limpía", se obtuvo a partir del notebook: 
- 01-Limpieza-Enriquecimiento.ipynb

**Información constante del dataset** <br>
- PAIS  - (País de origen)
- SEXO	
- EDAD - (la edad en el año que recibió el benefício)
- ESTUDIOS	
- ACTIVIDAD	
- PROFESION
- BENEFICIO - (Visa temporária o definitiva)
- TIT_DEP - (El que recibe el benefício es Titular o Dependente)
- ANIO - (año en que recibió el benefício)
- COMUNA

---

In [2]:
!ls ./data

01_extranjeros_rm.xlsx                PDs_2018_1.xlsx
CENSO_2017_COMUNAS_RM.xlsx            Visas-2017.xlsx
Comunas_RM_Mapas_Vectoriales          Visas-2018_1.xlsx
ML_extranjeros_rm.xlsx                censo_2017_RM.xlsx
PDs-2005-2016.xlsx                    censo_2017_inmigrantes_rm_comuna.xlsx
PDs-2017.xlsx


In [3]:
df = pd.read_excel('./data/01_extranjeros_rm.xlsx')
df.shape

(589106, 10)

In [4]:
df.head(2)

,PAIS,SEXO,EDAD,ESTUDIOS,ACTIVIDAD,PROFESION,BENEFICIO,TIT_DEP,ANIO,COMUNA
0,peru,femenino,32,no informa,empleado,matrona,definitiva,t,2006,santiago
1,ecuador,masculino,58,no informa,empleado,ingeniero,definitiva,t,2007,providencia


### 2. Reducir el dadaset a titulares mayores que 18 años y menores que 85

`La idea es trabajar con los extranjeros que llegan al pais que tengan poder de decision a cerca de donde van a vivir. Para esto, basado en las análisis de los datos (notebook '02-Analisis-Datos.ipynb'), haremos un filtro en titulares entre 18 y 85 años. No se hará distinción entre los diferentes benefícios: Definitivo y/o Temporário.`

In [5]:
df = df[(df.TIT_DEP == 't') & (df.EDAD >=18) & (df.EDAD < 85)]

In [6]:
df.reset_index(drop = True, inplace=True)
df.head(2)

,PAIS,SEXO,EDAD,ESTUDIOS,ACTIVIDAD,PROFESION,BENEFICIO,TIT_DEP,ANIO,COMUNA
0,peru,femenino,32,no informa,empleado,matrona,definitiva,t,2006,santiago
1,ecuador,masculino,58,no informa,empleado,ingeniero,definitiva,t,2007,providencia


In [7]:
len(df.PROFESION.unique())

613

`Como profesion, se observa que hay un numero muy grande de diferentes valores (613), por lo que denota que no existe critério para la captura de este dato. Esta característica, dada su disperción, no nos parece útil en la preparación de la data para ML y deberá ser deletada.`

### 3. Selecionar las columnas de interes

In [8]:
df = df[['PAIS', 'SEXO', 'EDAD', 'ESTUDIOS', 'ACTIVIDAD', 'ANIO', 'COMUNA']]

In [9]:
df.head()

,PAIS,SEXO,EDAD,ESTUDIOS,ACTIVIDAD,ANIO,COMUNA
0,peru,femenino,32,no informa,empleado,2006,santiago
1,ecuador,masculino,58,no informa,empleado,2007,providencia
2,brasil,masculino,27,no informa,empresario,2008,las condes
3,bolivia,femenino,55,no informa,empresario,2007,lampa
4,el salvador,femenino,41,no informa,inactivo,2007,santiago


### 4. Reducir el numero de categorias encontrado en PAIS. 
`Definamos 20 categorías para pais de origen, definindo labels especificos que contengan 97% extranjeros y un label "otros'`

In [10]:
def simplificaPais(text):
    text = text.split(' ')
    if 'peru' in text: text = 'peru'
    elif 'venezuela' in text: text= 'venezuela'
    elif 'haiti' in text: text= 'haiti'
    elif 'colombia' in text: text= 'colombia'
    elif 'bolivia' in text: text= 'bolivia'
    elif 'argentina' in text: text= 'argentina'
    elif 'ecuador' in text: text= 'ecuador'
    elif 'china' in text: text= 'china'
    elif "espana" in text: text= "espana"
    elif 'brasil' in text: text= 'brasil'
    elif 'dominicana' in text: text= 'dominicana'
    elif 'cuba' in text: text= 'cuba'
    elif 'estados' in text: text= 'eua'
    elif 'uruguay' in text: text= 'uruguay'
    elif 'mexico' in text: text= 'mexico'
    elif 'francia' in text: text= 'francia'
    elif 'paraguay' in text: text= 'paraguay'
    elif 'alemania' in text: text= 'alemania'
    elif 'italia' in text: text= 'italia'
    else: text = 'otros'
    return text 

In [11]:
df['PAIS'] = df.PAIS.apply(simplificaPais)

In [12]:
df.PAIS.value_counts()

peru          133874
venezuela     108842
haiti          77039
colombia       60867
bolivia        16161
argentina      15940
ecuador        15048
otros          13406
china           8736
espana          8182
brasil          7042
dominicana      5909
cuba            5458
eua             3914
uruguay         2898
mexico          2828
francia         2440
paraguay        1674
alemania        1565
italia          1541
Name: PAIS, dtype: int64

### 5. Converter variables continuas en categoricas

In [13]:
df['EDAD'].describe()

count    493364.000000
mean         34.019740
std          10.150718
min          18.000000
25%          27.000000
50%          32.000000
75%          39.000000
max          84.000000
Name: EDAD, dtype: float64

`En terminos de edad, lo que se procura acá es dividir los extranjeros entre los jovenes que vienen a procura de empezar su vida profesional; los adultos que ya llegan con una vida y/o profesion consolidada y los jubilados.`

In [14]:
df['EDAD'] = pd.cut(df['EDAD'], 
                      bins = [17, 39, 65, 120],
                      labels = ['18-39', '40-64', '65+'])

In [15]:
df.EDAD.value_counts()

18-39    373471
40-64    115014
65+        4879
Name: EDAD, dtype: int64

`En relación al año en que el extranjero obtuvo su benefício se considera los extranjeros que llegaron pré/pós terremoto de 2010 y los del "boom" inmigratório de los últimos años.`

In [16]:
df['ANIO'] = pd.cut(df['ANIO'], 
                      bins = [2004, 2010, 2017, 2019],
                      labels = ['2005-09', '2010-16', '2017+'])

In [17]:
df.ANIO.value_counts()

2010-16    303863
2017+      126109
2005-09     63392
Name: ANIO, dtype: int64

In [18]:
df.sample(5)

,PAIS,SEXO,EDAD,ESTUDIOS,ACTIVIDAD,ANIO,COMUNA
266951,espana,masculino,40-64,universitario,empleado,2010-16,providencia
304902,venezuela,femenino,18-39,medio,empleado,2010-16,nunoa
146016,argentina,femenino,18-39,no informa,inactivo,2010-16,las condes
81171,francia,masculino,18-39,universitario,empleado,2005-09,vitacura
83370,espana,masculino,40-64,tecnico,inactivo,2010-16,la florida


### 6. Reducir el numero de clases para COMUNA. 

`COMUNA será nuestra variable dependente (y),pues la idea es predicir donde vivería un extranjero que llega a región Metropoitana. La RM posue 52 comunas, las cuales deberán ser agrupadas en clusters (sectores). En la análisis de los datos se observó que en general los extranjeros se quedan en subregiones, por lo tanto reduciremos COMUNA a 6 clases:`
- centro
- poniente 
- oriente 
- sureste
- sur
- norte

**El mapa abajo, muestra como estarán distribuídas las comunas:**

<p><img src="./images/RM_comunas.png"></p>

In [19]:
def clusterRM(text):
    centro = ['santiago']
    oriente = ['providencia', 'vitacura', 'lo barnechea', 'las condes', 'la reina', 'nunoa']
    sureste = ['san jose de maipo', 'macul', 'la florida', 'penalolen',]
    sur = ['alhue', 'calera de tango', 'buin', 'isla de maipo', 'el bosque', 'paine', 'la granja', 'pedro aguirre cerda', 'lo espejo', 'puente alto', 'san joaquin', 'san miguel', 'pirque', 'san bernardo', 'san ramon', 'la cisterna', 'talagante', 'la pintana', ]
    norte = ['tiltil', 'renca', 'quinta normal', 'cerro navia', 'lampa', 'conchali', 'recoleta', 'independencia', 'colina', 'huechuraba', 'quilicura', ]
    poniente = ['san pedro', 'maria pinto', 'el monte', 'curacavi', 'padre hurtado', 'melipilla', 'estacion central', 'maipu', 'lo prado', 'pudahuel', 'penaflor', 'cerrillos', ]
    if   text in centro:  text = 'centro'
    elif text in oriente: text = 'oriente'
    elif text in sureste: text = 'sureste'
    elif text in sur:     text = 'sur'
    elif text in norte:   text = 'norte'
    elif text in poniente: text = 'poniente'
    else: text = 'otras'
    return text 

In [20]:
data = df.copy()

In [21]:
data['COMUNA'] = data.COMUNA.apply(clusterRM)

In [22]:
data.COMUNA.value_counts()

centro      139020
norte       116910
oriente      83530
sur          64173
poniente     63063
sureste      26668
Name: COMUNA, dtype: int64

In [23]:
data.sample(5)

,PAIS,SEXO,EDAD,ESTUDIOS,ACTIVIDAD,ANIO,COMUNA
413029,venezuela,masculino,18-39,medio,empleado,2017+,centro
47448,peru,femenino,40-64,medio,domestico,2005-09,sur
488717,peru,femenino,40-64,medio,empleado,2017+,centro
466391,haiti,masculino,18-39,medio,obrero,2017+,sur
462661,haiti,femenino,18-39,no informa,empleado,2017+,sur


In [24]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 493364 entries, 0 to 493363
Data columns (total 7 columns):
PAIS         493364 non-null object
SEXO         493364 non-null object
EDAD         493364 non-null category
ESTUDIOS     493364 non-null object
ACTIVIDAD    493364 non-null object
ANIO         493364 non-null category
COMUNA       493364 non-null object
dtypes: category(2), object(5)
memory usage: 19.8+ MB


### 7. Codificar variable target (COMUNA) con Label Encoder

In [25]:
com = LabelEncoder() 
data['COMUNA'] = com.fit_transform(data.COMUNA)

In [26]:
data.head()

,PAIS,SEXO,EDAD,ESTUDIOS,ACTIVIDAD,ANIO,COMUNA
0,peru,femenino,18-39,no informa,empleado,2005-09,0
1,ecuador,masculino,40-64,no informa,empleado,2005-09,2
2,brasil,masculino,18-39,no informa,empresario,2005-09,2
3,bolivia,femenino,40-64,no informa,empresario,2005-09,1
4,otros,femenino,40-64,no informa,inactivo,2005-09,0


### 8. Codificar las demás variables categoricas como numéricas

`La variable SEXO será codificada como numerica binária, con "Label Encoder".` 

In [27]:
sex = LabelEncoder() 
data['SEXO'] = sex.fit_transform(data.SEXO)

`Para las demás, como la ordenen no tiene mucho sentido, se trabajará con la técnica de "One Hot Encoder".`

In [28]:
data = pd.concat([data, pd.get_dummies(data.PAIS)], axis = 1)

In [29]:
data = pd.concat([data, pd.get_dummies(data.EDAD)], axis = 1)
data = pd.concat([data, pd.get_dummies(data.ESTUDIOS)], axis = 1)
data = pd.concat([data, pd.get_dummies(data.ACTIVIDAD)], axis = 1)
data = pd.concat([data, pd.get_dummies(data.ANIO)], axis = 1)

`Las columnas originales deben ser deletadas, como también la columna 'otros' que fue generada, pues en caso de que un extranjero no sea de ningún de los 19 paises (todas columnas en cero), el será obrigatóriamente de 'otros'` 

In [30]:
del data['PAIS']
del data['EDAD']
del data['ESTUDIOS']
del data['ACTIVIDAD']
del data['ANIO']

In [31]:
del data['otros']

In [32]:
data.head()

,SEXO,COMUNA,alemania,argentina,bolivia,brasil,china,colombia,cuba,dominicana,...,empresario,estudiante,inactivo,jubilado,no_informa,obrero,religioso,2005-09,2010-16,2017+
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,1,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,1,2,0,0,0,1,0,0,0,0,...,1,0,0,0,0,0,0,1,0,0
3,0,1,0,0,1,0,0,0,0,0,...,1,0,0,0,0,0,0,1,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,1,0,0


`En este punto, el dataset ya está listo para ser trabajado por los algoritimos de Machine Learning. El dataset limpio y preparado ("ML_extranjeros_rm.xlsx") será entonces salvado para posterio uso:`

In [33]:
data.to_excel('./data/ML_extranjeros_rm.xlsx')

### 9. Observaciónes finales

`En el proyecto de Machine Learning, una vez que se trabajará con modelos de clasificación supervisionados, el dataset deberá ser separado en un array 'X' con las características y un vector 'y' con la variable target.`

In [34]:
X = data.copy()
y = X.pop('COMUNA').values

In [35]:
y

array([0, 2, 2, ..., 1, 1, 1])

In [36]:
X.head(2)

,SEXO,alemania,argentina,bolivia,brasil,china,colombia,cuba,dominicana,ecuador,...,empresario,estudiante,inactivo,jubilado,no_informa,obrero,religioso,2005-09,2010-16,2017+
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,1,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,0


`En se tratando de clasificación, en el trabajo de ML, se hará testes con los seguintes modelos:`

- Naive Bayes
- Decision Trees
- Randon Forest
- Logistic Regression
- Linear support vector Classificator

---